### 8. Game Play Analysis I Write a solution to find the first login date for each player. Return the result table in any order.

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *  # Import the function
from pyspark.sql.functions import regexp_replace, col
from pyspark.sql.functions import col, explode, array_repeat
from pyspark.sql.window import Window
import json
spark = SparkSession.builder.getOrCreate()

In [2]:
# Define the schema for the "Activity"
activity_schema = StructType([
StructField("player_id", IntegerType(), True),
StructField("device_id", IntegerType(), True),
StructField("event_date", StringType(), True),
StructField("games_played", IntegerType(), True)
])
# Define data for the "Activity"
activity_data = [
(1, 2, '2016-03-01', 5),
(1, 2, '2016-05-02', 6),
(2, 3, '2017-06-25', 1),
(3, 1, '2016-03-02', 0),
(3, 4, '2018-07-03', 5)
]

# Create a PySpark DataFrame
activity_df = spark.createDataFrame(activity_data, activity_schema)

activity_df.show()



+---------+---------+----------+------------+
|player_id|device_id|event_date|games_played|
+---------+---------+----------+------------+
|        1|        2|2016-03-01|           5|
|        1|        2|2016-05-02|           6|
|        2|        3|2017-06-25|           1|
|        3|        1|2016-03-02|           0|
|        3|        4|2018-07-03|           5|
+---------+---------+----------+------------+



In [3]:
w = Window.partitionBy('player_id').orderBy('event_date')
rnk_df = activity_df.withColumn('rnk', rank().over(w))
rnk_df.show()

+---------+---------+----------+------------+---+
|player_id|device_id|event_date|games_played|rnk|
+---------+---------+----------+------------+---+
|        1|        2|2016-03-01|           5|  1|
|        1|        2|2016-05-02|           6|  2|
|        2|        3|2017-06-25|           1|  1|
|        3|        1|2016-03-02|           0|  1|
|        3|        4|2018-07-03|           5|  2|
+---------+---------+----------+------------+---+



In [4]:
rnk_1_df = rnk_df.filter(col('rnk') == 1)
rnk_1_df.show()

+---------+---------+----------+------------+---+
|player_id|device_id|event_date|games_played|rnk|
+---------+---------+----------+------------+---+
|        1|        2|2016-03-01|           5|  1|
|        2|        3|2017-06-25|           1|  1|
|        3|        1|2016-03-02|           0|  1|
+---------+---------+----------+------------+---+



In [5]:
result = rnk_1_df.select(col('player_id'),col('device_id'),col('event_date'))
result.show()


+---------+---------+----------+
|player_id|device_id|event_date|
+---------+---------+----------+
|        1|        2|2016-03-01|
|        2|        3|2017-06-25|
|        3|        1|2016-03-02|
+---------+---------+----------+

